<a href="https://colab.research.google.com/github/jung-geun/PSO/blob/master/example/pso2mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본 설치

아래 명령어를 통해 설치할 수 있습니다
```python
!pip install pso2keras
```

필수 패키지로 tensorflow 가 필요하며, log 분석시에는 tensorboard 가 추가로 필요합니다

In [ ]:
import sys
print('python version ', sys.version)

# !pip uninstall pso2keras
!pip install --upgrade pip
!pip install pso2keras

# 모델 생성

keras 모델을 사용하여 학습하기 때문에 모델을 생성하여 입력을 해주어야 합니다.

In [2]:
import os
import sys

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import gc

import tensorflow as tf
from keras.datasets import mnist
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from tensorflow import keras

from pso import Optimizer


def get_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.reshape((60000, 28, 28, 1))
    x_test = x_test.reshape((10000, 28, 28, 1))

    y_train, y_test = tf.one_hot(y_train, 10), tf.one_hot(y_test, 10)

    x_train, x_test = tf.convert_to_tensor(x_train), tf.convert_to_tensor(x_test)
    y_train, y_test = tf.convert_to_tensor(y_train), tf.convert_to_tensor(y_test)

    print(f"x_train : {x_train[0].shape} | y_train : {y_train[0].shape}")
    print(f"x_test : {x_test[0].shape} | y_test : {y_test[0].shape}")

    return x_train, y_train, x_test, y_test


def get_data_test():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_test = x_test / 255.0
    x_test = x_test.reshape((10000, 28, 28, 1))

    y_test = tf.one_hot(y_test, 10)

    x_test = tf.convert_to_tensor(x_test)
    y_test = tf.convert_to_tensor(y_test)

    print(f"x_test : {x_test[0].shape} | y_test : {y_test[0].shape}")

    return x_test, y_test


def make_model():
    model = Sequential()
    model.add(
        Conv2D(32, kernel_size=(5, 5), activation="relu", input_shape=(28, 28, 1))
    )
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))
    model.add(Dense(10, activation="softmax"))

    return model

# 학습

학습을 위한 particle 개수와 기본 설정이 필요합니다
loss 는 tensorflow 의 loss 를 활용합니다

In [ ]:
model = make_model()
x_train, y_train = get_data_test()

loss = 'mean_squared_error'

pso_mnist = Optimizer(
    model,
    loss=loss,
    n_particles=500,
    c0=0.35,
    c1=0.8,
    w_min=0.6,
    w_max=1.2,
    negative_swarm=0.1,
    mutation_swarm=0.2,
    particle_min=-5,
    particle_max=5,
)

best_score = pso_mnist.fit(
    x_train,
    y_train,
    epochs=200,
    save_info=True,
    log=2,
    log_name="mnist",
    save_path="./result/mnist",
    renewal="acc",
    check_point=25,
)

print("Done!")

11490434/11490434 [==============================] - 0s 0us/step
x_test : (28, 28, 1) | y_test : (10,)
start running time : 20230723-095443


Initializing Particles:   0%|          | 0/500 [00:00<?, ?it/s]

negative swarm : 50 / 500
mutation swarm : 20.0%


Initializing velocity:   0%|          | 0/500 [00:00<?, ?it/s]

initial g_best_score : 0.2117999941110611


best 0.2118|0.1576:   0%|          | 0/200 [00:00<?, ?it/s]

acc : 0.0000 loss : inf:   0%|          | 0/500 [00:00<?, ?it/s]

acc : 0.0000 loss : inf:   0%|          | 0/500 [00:00<?, ?it/s]

acc : 0.0000 loss : inf:   0%|          | 0/500 [00:00<?, ?it/s]